<a href="https://colab.research.google.com/github/Rahee93/DataEngineer_Module_5/blob/main/DataEngineer_Module_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet"
response = requests.get(url)

with open("yellow_tripdata_2024-10.parquet", "wb") as file:
    file.write(response.content)

print("Download complete!")


Download complete!


In [2]:
## To install yellow tripdata on my local
from google.colab import files
files.download('yellow_tripdata_2024-10.parquet')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
import pandas as pd

# Read the Parquet file
df = pd.read_parquet('yellow_tripdata_2024-10.parquet')

# Print the DataFrame
print(df)


         VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0               2  2024-10-01 00:30:44   2024-10-01 00:48:26              1.0   
1               1  2024-10-01 00:12:20   2024-10-01 00:25:25              1.0   
2               1  2024-10-01 00:04:46   2024-10-01 00:13:52              1.0   
3               1  2024-10-01 00:12:10   2024-10-01 00:23:01              1.0   
4               1  2024-10-01 00:30:22   2024-10-01 00:30:39              1.0   
...           ...                  ...                   ...              ...   
3833766         2  2024-10-31 23:49:01   2024-11-01 00:04:31              NaN   
3833767         2  2024-10-31 23:35:15   2024-10-31 23:52:50              NaN   
3833768         2  2024-10-31 23:30:43   2024-11-01 00:08:12              NaN   
3833769         2  2024-10-31 23:00:00   2024-10-31 23:06:00              NaN   
3833770         2  2024-10-31 23:18:00   2024-10-31 23:51:00              NaN   

         trip_distance  Rat

In [4]:
import pandas as pd

# Load the Parquet file into a DataFrame
df = pd.read_parquet('yellow_tripdata_2024-10.parquet')

# Check the first few rows to confirm the column names
print(df.head())

# Convert the 'tpep_pickup_datetime' column to datetime
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])

# Filter trips that started on the 15th of October, 2024
oct_15_trips = df[df['tpep_pickup_datetime'].dt.date == pd.to_datetime('2024-10-15').date()]

# Count the number of trips
num_trips = oct_15_trips.shape[0]
print(f"Number of trips on 15th October 2024: {num_trips}")



   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2024-10-01 00:30:44   2024-10-01 00:48:26              1.0   
1         1  2024-10-01 00:12:20   2024-10-01 00:25:25              1.0   
2         1  2024-10-01 00:04:46   2024-10-01 00:13:52              1.0   
3         1  2024-10-01 00:12:10   2024-10-01 00:23:01              1.0   
4         1  2024-10-01 00:30:22   2024-10-01 00:30:39              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0            3.0         1.0                  N           162           246   
1            2.2         1.0                  N            48           236   
2            2.7         1.0                  N           142            24   
3            3.1         1.0                  N           233            75   
4            0.0         1.0                  N           262           262   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [5]:
# Question 4: Longest trip
# What is the length of the longest trip in the dataset in hours?
# Convert 'tpep_pickup_datetime' and 'tpep_dropoff_datetime' to datetime
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

# Calculate the trip duration in hours
df['trip_duration_hours'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 3600

# Find the maximum trip duration
longest_trip_duration = df['trip_duration_hours'].max()
print(f"The length of the longest trip is: {longest_trip_duration} hours")

The length of the longest trip is: 162.61777777777777 hours


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Least Frequent Pickup Location Zone") \
    .getOrCreate()



In [7]:
# Download taxi zone lookup data
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

# Load the taxi zone lookup data (zone lookup data CSV)
zone_data_path = "./taxi_zone_lookup.csv"  # Path to the downloaded file
zone_df = spark.read.csv(zone_data_path, header=True, inferSchema=True)

zone_df.show(5)

--2025-03-06 17:09:02--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.84.160.73, 52.84.160.213, 52.84.160.116, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.84.160.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-03-06 17:09:03 (51.7 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet 

In [8]:
# Check the column names of the DataFrames
df.columns
zone_df.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [9]:
# Question 6: Least frequent pickup location zone
# Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?
from pyspark.sql import SparkSession
import pandas as pd

In [10]:
# Initialize Spark session
spark = SparkSession.builder.appName("TaxiAnalysis").getOrCreate()

In [11]:
# Convert the Pandas DataFrame to Spark DataFrame
yellow_taxi_data = spark.createDataFrame(df)


In [12]:
# Create a temporary view for the yellow_taxi_data
yellow_taxi_data.createOrReplaceTempView("yellow_taxi_data")

In [13]:
# Assuming zone_lookup DataFrame is already created and available as zone_df
zone_df.createOrReplaceTempView("zone_lookup")

In [14]:
# Execute the query to find least frequent pickup location zone
query = """
SELECT z.Zone, COUNT(1) AS trip_count
FROM yellow_taxi_data y
JOIN zone_lookup z ON y.PULocationID = z.LocationID
GROUP BY z.Zone
ORDER BY trip_count ASC
LIMIT 1
"""

In [29]:
# Load the taxi zone lookup data
zone_df = spark.read.csv("./taxi_zone_lookup.csv", header=True, inferSchema=True)
zone_df.createOrReplaceTempView("zone_lookup")



In [30]:
# Load the Yellow October 2024 data
yellow_october_df = spark.read.csv("yellow_tripdata_2024-10.parquet", header=True, inferSchema=True)
yellow_october_df.createOrReplaceTempView("yellow_october")

In [20]:
# Run the query
least_frequent_zone = spark.sql(query)

In [ ]:
# Show the least frequent zone
least_frequent_zone.show()